In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def rbf(x1, x2, gamma=0.1):
    return np.exp(-gamma*np.linalg.norm(x1-x2)**2)

## Реализация KRLS

In [ ]:
from scipy.linalg import block_diag

class KRLS:
    def clear(self):
        self.K = None
        self.invK = None
        self.alpha = None
        self.P = None
        self.D = []

    def __init__(self, v = 0.5, kernel=rbf):
        self.kernel = kernel
        self.v = v
        self.clear()
    
    def fit(self, X, y):
        if (X.shape[0] != len(y)):
            raise ValueError("Dimensions error")
        
        y = y.reshape(len(y), 1)
        
        start = 0
        if self.K is None:
            self.K = np.array([[self.kernel(X[0], X[0])]])
            self.invK = np.array([[1 / self.K[0][0]]])
            self.alpha = np.array([y[0] / self.K[0][0]]).reshape(1, 1)
            self.P = np.array([[1]])
            self.D.append(X[0].reshape(1, X.shape[1]))
            start = 1
        
        for t in range(start, len(y)):
            xt = X[t].reshape(X.shape[1], 1)
            kt = np.array([self.kernel(xi, xt) for xi in self.D]).reshape(len(self.D), 1)
            at = self.invK.dot(kt)
            delta = self.kernel(xt, xt) - np.vdot(kt.T, at)
            if delta > self.v:
                self.D.append(xt)
                self.invK = 1 / delta * np.block([[delta*self.invK + at.dot(at.T), -at], [-at.T, 1]])
                at = np.array([0]*(len(self.D) - 2)+[1]).reshape(len(self.D)-1, 1)
                self.P = block_diag(self.P, 1)
                tmp = 1 / delta * (y[t].squeeze() - np.vdot(kt.T, self.alpha))
                self.alpha = np.vstack((self.alpha - tmp*at, tmp))
            else:
                qt = self.P.dot(at) / (1 + at.T.dot(self.P).dot(at)).squeeze()
                self.P = self.P - qt.dot(at.T).dot(self.P)
                self.alpha = self.alpha + self.invK.dot(qt)*(y[t].squeeze() - np.vdot(kt.T, self.alpha))
        
        self.points = [self.D[i][0][0] for i in range(len(self.D))]
                
    
    def predict(self, x):
        y = 0
        for a, xi in zip(self.alpha, self.D):
            y += a * self.kernel(x, xi)
        return y

## Точные данные

In [ ]:
n = 100
X = np.linspace(-2*np.pi, 2*np.pi, n).reshape(n, 1)
y = np.sin(X)
points_x = np.linspace(-2*np.pi, 2*np.pi, 250)
points_y = np.sin(points_x)

In [ ]:
vs = np.linspace(0.01, 0.5, 100)
gammas = [0.38846153846153847]
random_states = [42]
min_err = np.inf
best = tuple()
errs = []
num_of_vectors = []
for random_state in random_states:
    np.random.seed(random_state)
    X = np.linspace(-2*np.pi, 2*np.pi, n).reshape(n, 1)
    np.random.shuffle(X)
    y = np.sin(X)
    for v in vs:
        for gamma in gammas:
            krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1, x2, gamma))
            krls.fit(X, y)
            predict_y = [krls.predict(x)[0] for x in points_x]
            err = mean_squared_error(points_y, predict_y, squared=False)
            errs.append(err)
            num_of_vectors.append(len(krls.points))
            if err < min_err:
                min_err = err
                best_pair = (random_state, v, gamma)

In [ ]:
plt.plot(num_of_vectors, errs)

In [ ]:
print(best_pair, min_err)

(42, 0.16794871794871796, 0.38846153846153847) 0.9498581724051856

In [ ]:
#best_pair = (42, 0.16794871794871796, 0.38846153846153847)

In [ ]:
random_state, v, gamma = best_pair
np.random.seed(random_state)
X = np.linspace(-2*np.pi, 2*np.pi, n).reshape(n, 1)
np.random.shuffle(X)
y = np.sin(X)
krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1, x2, gamma))
krls.fit(X, y)

In [ ]:
plt.plot(points_x, points_y)
plt.plot(X, y, '.')
plt.plot(krls.points, np.sin(krls.points), '*', color='blue')
predict_y = [krls.predict(x)[0] for x in points_x]
plt.plot(points_x, [krls.predict(xj)[0] for xj in points_x])
plt.ylim(-1.5, 1.5)
plt.grid()

## Зашумлённые данные

In [ ]:
y_noise = np.array([x[0] + np.random.normal(0, 0.2) for x in np.sin(X)]).reshape(len(X), 1)

In [ ]:
vs = np.linspace(0, 1, 20)
gammas = np.linspace(0, 1, 20)
min_err = np.inf
best_pair = tuple()
for v in vs:
    for gamma in gammas:
        krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1,x2, gamma))
        krls.fit(X, y_noise)
        predict_y = [krls.predict(x)[0] for x in points_x]
        err = np.linalg.norm(predict_y - points_y, ord=2)
        if err < min_err:
            min_err = err
            best_pair = (v, gamma)

In [ ]:
v, gamma = best_pair
krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1,x2, gamma))
krls.fit(X, y_noise)

In [ ]:
print(best_pair, min_err)

In [ ]:
plt.plot(points_x, points_y, linestyle = '--')
plt.plot(X, y_noise, '.')
plt.plot(krls.points, np.sin(krls.points), '*', color='blue')
predict_y = [krls.predict(x)[0] for x in points_x]
plt.plot(points_x, [krls.predict(xj)[0] for xj in points_x])
plt.grid()

## Real world dataset

In [ ]:
X, y = load_boston(return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [ ]:
vs = np.linspace(0.1, 0.5, 30)
gammas = np.linspace(0.05, 0.2, 10)

In [ ]:
min_err = np.inf
best_pair = tuple()
i = 0
for v in vs:
    i += 1
    for gamma in gammas:
        krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1,x2, gamma))
        krls.fit(X_train, y_train)
        y_predict = np.array([krls.predict(x) for x in X_test])
        err = mean_squared_error(y_test, y_predict, squared=False)
        if err < min_err:
            min_err = err
            best_pair = (v, gamma)
    print(i)

In [ ]:
print(best_pair, min_err)

In [ ]:
v, gamma = best_pair
krls = KRLS(v=v, kernel=lambda x1, x2: rbf(x1,x2, gamma))
krls.fit(X, y)

In [ ]:
len(krls.D)

In [ ]:
np.mean(y)

In [ ]:
np.sqrt(np.linalg.norm(y_test - np.mean(y_train))**2 / len(y_test))